# MOwNiT - lab 8
## PageRank

### Essential imports

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import snap
import heapq
import os

### Loading SNAP PNGraph from file and converting to networkx.DiGraph

In [ ]:
def load_snap(name):
    if not os.path.isfile(name):
        raise FileNotFoundError
    return snap.LoadEdgeList(snap.PNGraph, name, 0, 1)

def nx_to_snap(G_nx):
    G_snap = snap.TNGraph.New()
    for node in G_nx.nodes():
        G_snap.AddNode(node)
    for (u,v) in G_nx.edges():
        G_snap.AddEdge(u,v)
    return G_snap

### Drawing utility
#### Drawing tool for ranked graph *draw_ranks*
Parameters:
- G: *networkx.DiGraph*
- ranks: *np.ndarray* calculated ranks of shape (N,1)
- layout: *networkx.DiGraph -> dict* networkx layout function taking graph as an only argument

In [ ]:
%matplotlib notebook

def draw_ranks(G, ranks, layout=nx.random_layout):
    plt.figure(figsize=(8,6))
    pos = layout(G)
    nx.draw_networkx(G, pos = pos, labels={i: f'{ranks[i,0]:.4f}'[1:] for i in G.nodes()}, node_size=1200)
    plt.show()

### Power iteration algorithm to find dominant eigenvector with Rayleigh quotient

In [ ]:
def rayleigh(A, v):
    Av = A @ v
    return v.T @ Av

def power_iteration(A, eps = 10e-20, max_iters = 100):
    N = A.shape[0]

    xn = np.ones((N, 1)) / np.sqrt(N)
    eigenvalue_new = rayleigh(A, xn)
    i = 0
    while True:
        i += 1
        x0 = xn
        eigenvalue = eigenvalue_new
        xn = A @ x0
        xn /= np.linalg.norm(xn)
        eigenvalue_new = rayleigh(A, xn)
        if np.abs(eigenvalue - eigenvalue_new) < eps or i >= max_iters:
            break
    return eigenvalue_new, xn / np.sum(np.abs(xn))

## Simple ranking algorithm
Parameters:
- G: *networkx.DiGraph*
- eps: *float* argument $\varepsilon$ for power_iteration

Returns:
- ranks: *numpy.ndarray* with calculated ranks of shape (N,1) 

In [ ]:
def simple_ranker(G):
    A = nx.adjacency_matrix(G).astype(np.float64)
    for node in G.nodes():
        out_deg = G.out_degree(node)
        if out_deg != 0:
            A[node,:] /= out_deg
    return power_iteration(A.T)[1]

## G_cycle
#### Simple directed cycle

## G_cycle_with_main
#### Directed cycle with one main sink

## G_heap_with_main
#### Structure of website with main site and subsites linking to main site

In [ ]:
#########
size = 30
#########

##################################

G_cycle = nx.DiGraph()
for i in range(size):
    G_cycle.add_node(i)

for i in range(size):
    G_cycle.add_edge(i, (i+1) % size)

###################################

G_cycle_with_main = nx.DiGraph()
for i in range(size):
    G_cycle_with_main.add_node(i)

for i in range(size):
    G_cycle_with_main.add_edge(i, (i+1) % size)

for i in range(1, size-1):
    G_cycle_with_main.add_edge(i, 0)

G_cycle_with_main.add_edge(2, 7)
G_cycle_with_main.add_edge(0, 7)

###################################

levels = 4

G_heap_with_main = nx.DiGraph()
G_heap_with_main.add_node(0)
for node_parent in range(2**levels - 1):
    for node_child in [node_parent*2 + 1, node_parent*2 + 2]:
        G_heap_with_main.add_node(node_child)
        G_heap_with_main.add_edge(node_parent, node_child)
        G_heap_with_main.add_edge(node_child, 0)

In [ ]:
%matplotlib notebook


print("Simple cycle")
r_cycle = simple_ranker(G_cycle)
draw_ranks(G_cycle, r_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
r_cycle_main = simple_ranker(G_cycle_with_main)
draw_ranks(G_cycle_with_main, r_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
r_heap = simple_ranker(G_heap_with_main)
draw_ranks(G_heap_with_main, r_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

## PageRank
Parameters:
- G: *snap.TNGraph*
- c: *float* indicating how important will adjacency matrix be in calculating PageRank in comparrasion to random jumps
- e: *numpy.ndarray* of probabilities of random jumps to nodes
- eps: *float* change of result that we consider small enough to stop calculations
- computed_graph_info: *tuple* information computed by compute_pagerank_graph_info(G)

Returns:
- ranks: *numpy.ndarray* with calculated ranks of shape (N,1) 

In [ ]:
def compute_pagerank_graph_info(G):
    # Map node number to index in matrix
    mapping = {}
    
    # Map index to node number
    inv_mapping = []

    for i, node in enumerate(G.Nodes()):
        mapping[node.GetId()] = i
        inv_mapping.append(node.GetId())

    # Getting number of nodes
    N = len(inv_mapping)
    
    # Getting out degrees of all nodes
    out_degs = np.zeros(N)
    for node in G.Nodes():
        out_degs[mapping[node.GetId()]] = node.GetOutDeg() 

    # Calculating transposed adjacency matrix
    A = sparse.lil_matrix((N,N), dtype=np.float32)
    for edge in G.Edges():
        A[mapping[edge.GetDstNId()], mapping[edge.GetSrcNId()]] = 1/out_degs[mapping[edge.GetSrcNId()]]
    
    return (A, N, mapping, inv_mapping)

def page_rank(G, c = 0.9, e=None, eps=10e-6, computed_graph_info=None):
    (A, N, mapping, inv_mapping) = compute_pagerank_graph_info(G) if computed_graph_info is None else computed_graph_info
    
    if e is None:
        e = np.ones((N,1), dtype=np.float32)/N
            
    B_A = c * A
    B_e = (1 - c) * e
    rn = np.ones((N,1), dtype=np.float32)/N
    while True:
        r0 = rn
        rn = B_A @ r0 + B_e * np.sum(r0)
        d = np.sum(np.abs(r0)) - np.sum(np.abs(rn))
        rn += d*e
        delta = np.sum(np.abs(rn - r0))
        if delta < eps:
            break
    return r0, mapping, inv_mapping

### PageRank testing

In [ ]:
print("Simple cycle")
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle))
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main))
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main))
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

### Influence of *c* parameter
Examples below show that by changing *c* we can tell PageRank algorithm how important adjacency matrix is for us. By setting *c* smaller, results get more and more similar to uniform distribution. It is caused by default, uniform distribution of *e*. In conclusion the smaller *c*, the more important random jumps are in algorithm. 

In [ ]:
# c = 0.7

print("Simple cycle")
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle), c=0.7)
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main), c=0.7)
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main), c=0.7)
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

In [ ]:
# c = 0.5

print("Simple cycle")
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle), c=0.5)
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main), c=0.5)
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main), c=0.5)
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

In [ ]:
# c = 0.01

print("Simple cycle")
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle), c=0.01)
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main), c=0.01)
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main), c=0.01)
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

### Influence of *e* parameter
Vector *e* tells likely it is to make jump to corresponding node. By manipulating it, algorithm can customize results of PageRank (the smaller parameter *c* is, the more visible customization we get). 

The best example here is simple cycle. By changing *c* parameter nothing changed in cycle's PageRank because all nodes had same parameters. Changing *e* for one of them, made it the most important node in our graph, algorithm started to prefer it when random jumps were made.

Another good example is setting *e[2]* 1000 times larger than other probabilities for heap with backedges to node 0. In this case one of root's child got PageRank score almost 2 times bigger than other child and so it's subtrees scores also increased.

In [ ]:
# e[0] - 50 times bigger 

print("Simple cycle")
N_cycle = len(list(G_cycle.nodes()))
e_cycle = np.ones((N_cycle,1))
e_cycle[0] = 50
e_cycle /= np.sum(np.abs(e_cycle))
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle), e = e_cycle)
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
N_cycle_with_main = len(list(G_cycle_with_main.nodes()))
e_cycle_with_main = np.ones((N_cycle_with_main,1))
e_cycle_with_main[0] = 50
e_cycle_with_main /= np.sum(np.abs(e_cycle_with_main))
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main), e = e_cycle_with_main)
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
N_heap_with_main = len(list(G_heap_with_main.nodes()))
e_heap_with_main = np.ones((N_heap_with_main,1))
e_heap_with_main[0] = 50
e_heap_with_main /= np.sum(np.abs(e_heap_with_main))
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main), e = e_heap_with_main)
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

In [ ]:
# e[2] - 1000 times bigger 

print("Simple cycle")
N_cycle = len(list(G_cycle.nodes()))
e_cycle = np.ones((N_cycle,1))
e_cycle[2] = 1000
e_cycle /= np.sum(np.abs(e_cycle))
pr_cycle, _, _ = page_rank(nx_to_snap(G_cycle), e = e_cycle)
draw_ranks(G_cycle, pr_cycle, layout=nx.circular_layout)

print("Cycle with main sink")
N_cycle_with_main = len(list(G_cycle_with_main.nodes()))
e_cycle_with_main = np.ones((N_cycle_with_main,1))
e_cycle_with_main[2] = 1000
e_cycle_with_main /= np.sum(np.abs(e_cycle_with_main))
pr_cycle_main, _, _ = page_rank(nx_to_snap(G_cycle_with_main), e=e_cycle_with_main)
draw_ranks(G_cycle_with_main, pr_cycle_main, layout=nx.circular_layout)

print("\"Heap\" with back edges")
N_heap_with_main = len(list(G_heap_with_main.nodes()))
e_heap_with_main = np.ones((N_heap_with_main,1))
e_heap_with_main[2] = 1000
e_heap_with_main /= np.sum(np.abs(e_heap_with_main))
pr_heap, _, _ = page_rank(nx_to_snap(G_heap_with_main), e = e_heap_with_main)
draw_ranks(G_heap_with_main, pr_heap, layout=lambda G: nx.drawing.nx_agraph.graphviz_layout(G, prog='dot'))

## Arxiv HEP-TH
Directed graph of high energy physics theory articles as nodes and citations between them as directed edges which were collected in years 1994-2003. Dataset statistics:
- 27,770 nodes 
- 352,807 edges

Data provided by SNAP database → https://snap.stanford.edu/data/cit-HepTh.html Text files with articles' basic information are also available there

In [ ]:
G_hep_cit = load_snap("Cit-HepTh.txt")
pr_hep_cit, hep_cit_mapping, hep_cit_inv_mapping = page_rank(G_hep_cit)

In [ ]:
N = len(hep_cit_inv_mapping)
hep_cit_biggest_ranks = list(
    map(
        lambda i: hep_cit_inv_mapping[i], 
        heapq.nlargest(
            10, 
            range(N), 
            key=lambda i: pr_hep_cit[i]
        )
    )
)

if os.path.isdir("cits_hep"):
    for number in hep_cit_biggest_ranks:
        print(f"{number} rank: {pr_hep_cit[hep_cit_mapping[number]]}")
        with open(f"cits_hep/{number}.abs", "r") as file:
            print(file.read())
        print("________________________________")
else:
    print(">>>\nTo print articles info \'cits_hep\' directory is needed with all *.abs files available at https://snap.stanford.edu/data/cit-HepTh.html\n>>>\n")
    for number in hep_cit_biggest_ranks:
        print(f"{number} rank: {pr_hep_cit[hep_cit_mapping[number]]}")
        print("________________________________")

## US Patents citation graph
Next example is directed graph of US patents as nodes and citations between them as directed edges which were collected in years 1975-1999. Dataset statistics:
- 3,774,768 nodes 
- 16,518,948 edges

Because of its volume it is not provided with project but it is available to download at https://snap.stanford.edu/data/cit-Patents.html. Precomputed results are under the source code.

In [ ]:
pat_graph_info = None
try:
    G_pat = load_snap("cit-Patents.txt")
    if pat_graph_info is None:
        pat_graph_info = compute_pagerank_graph_info(G_pat)
except FileNotFoundError:
    print("You need cit-Patents.txt from https://snap.stanford.edu/data/cit-Patents.html to proceed")

In [ ]:
try:
    if pat_graph_info is None:
        raise FileNotFoundError
    pr_pat, pat_mapping, pat_inv_mapping = page_rank(G_pat, computed_graph_info=pat_graph_info)
    N = len(pat_inv_mapping)
    pat_biggest_ranks = list(
        map(
            lambda i: pat_inv_mapping[i], 
            heapq.nlargest(
                10, 
                range(N), 
                key=lambda i: pr_pat[i]
            )
        )
    )
    for number in pat_biggest_ranks:
        print(f"{number} rank: {pr_pat[pat_mapping[number]]}")
        print(f"https://patents.google.com/patent/US{number}A/")
        print("________________________________")
except FileNotFoundError:
    print("You need cit-Patents.txt from https://snap.stanford.edu/data/cit-Patents.html to proceed")

### Top 10 patents by PageRank
- Rank: 9.78e-05
    - https://patents.google.com/patent/US4237224A/

- Rank: 8.94e-05
    - https://patents.google.com/patent/US3813316A/

- Rank: 5.30e-05
    - https://patents.google.com/patent/US3932805A/

- Rank: 4.31e-05
    - https://patents.google.com/patent/US4395486A/

- Rank: 4.17e-05
    - https://patents.google.com/patent/US4298685A/

- Rank: 3.89e-05
    - https://patents.google.com/patent/US3778614A/

- Rank: 3.74e-05
    - https://patents.google.com/patent/US4683195A/

- Rank: 3.42e-05
    - https://patents.google.com/patent/US4683202A/

- Rank: 3.40e-05
    - https://patents.google.com/patent/US3789832A/

- Rank: 3.40e-05
    - https://patents.google.com/patent/US3950357A/